In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../data/ReportsDATASET.csv')

In [ ]:
df

In [ ]:
df['Text'][0]

## Run llama 3 locally

- install `ollama`
- run `ollama pull llama3` to pull down the llama 3 8B model 
- start the model running using `ollama run llama3`

In [ ]:
import requests
import json

In [ ]:
url = "http://localhost:11434/api/chat"

In [ ]:
def llama3(prompt: str):
    data = {
        "model": "llama3",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "stream": False
    }
    
    headers = {
        'Content-Type': 'application/json'
    }
    
    response = requests.post(url, headers=headers, json=data)
    
    return response.json()['message']['content']

In [ ]:
response = llama3("who wrote the book the godfather?")

In [ ]:
response